In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import requests

NGROK_DOMAIN = "http://eez115.ece.ust.hk:5000/"
# NGROK_DOMAIN = "http://localhost:8080/"
session_id = 494016298


response = requests.post(f"{NGROK_DOMAIN}/get_paf_result", json={"session_id": session_id})
response = response.json()
response_data = response.get("responses", {})
conversation_history = response_data.pop("conversation_history", None)
table = pd.DataFrame.from_dict(response_data, orient='index')

In [3]:
conversation_history

[{'role': 'Grace', 'utterance': '你好啊！我喺 Grace。你今日覺得點啊？'},
 {'role': 'Patient', 'utterance': '好呀'},
 {'role': 'Grace', 'utterance': '請問你有冇咳？'},
 {'role': 'Patient', 'utterance': '冇呀'},
 {'role': 'Grace', 'utterance': '冇咳。有冇痰呀？'},
 {'role': 'Patient', 'utterance': '冇尋'},
 {'role': 'Grace', 'utterance': '你喺過去三個月有冇無緣無故變瘦？'},
 {'role': 'Patient', 'utterance': '冇呀'},
 {'role': 'Grace', 'utterance': '胃口點呀？有冇唔想食嘢？'},
 {'role': 'Patient', 'utterance': '冇呀\n你好唔好'},
 {'role': 'Grace', 'utterance': '好嘅。你有冇特別嘅嘢唔食㗎？ 例如豬，牛，雞？ 又或者你係食素嘅？'},
 {'role': 'Patient', 'utterance': '冇呀冇'},
 {'role': 'Grace', 'utterance': '好多謝你嘅時間，我已經將你所有嘅答案記錄曬。'},
 {'role': 'Patient', 'utterance': 'please repeat'}]

In [4]:
table

,patient_answer,question_asked,session_id,slot_name,slot_value
q1_0_coughing_existance,冇呀,請問你有冇咳？,494016298,cough_existence,no
q2_0_sputum_existance,冇尋,有冇痰呀？,494016298,sputum_existence,no
q5_0_lose_weight_existance,冇呀,你喺過去三個月有冇無緣無故變瘦？,494016298,weight_loss_existance,no
q5_2_decreased_appetite,冇呀\n你好唔好,胃口點呀？有冇唔想食嘢？,494016298,appetite_loss,no
q7_2_food_preference,冇呀冇,你有冇特別嘅嘢唔食㗎？ 例如豬，牛，雞？ 又或者你係食素嘅？,494016298,food_preference,no preference


In [5]:
table.reset_index(drop=True)[["session_id", "slot_name", "question_asked","slot_value"]]

,session_id,slot_name,question_asked,slot_value
0,494016298,cough_existence,請問你有冇咳？,no
1,494016298,sputum_existence,有冇痰呀？,no
2,494016298,weight_loss_existance,你喺過去三個月有冇無緣無故變瘦？,no
3,494016298,appetite_loss,胃口點呀？有冇唔想食嘢？,no
4,494016298,food_preference,你有冇特別嘅嘢唔食㗎？ 例如豬，牛，雞？ 又或者你係食素嘅？,no preference


In [6]:
table.slot_name.unique()

array(['cough_existence', 'sputum_existence', 'weight_loss_existance',
       'appetite_loss', 'food_preference'], dtype=object)

In [7]:
from fill_PAF_table import fill_paf_table

tab = fill_paf_table(response.get("responses", {}))
tab

,PAF_Question,PAF_Result
1,Coughing 請問有冇咳？,☑️ No \t ☐ Yes
2,Sputum 請問有冇痰？,☑️ No \t ☐ Yes
2.1,\t – \t Sputum Amount 有痰的話，有幾多量？,N.A.
2.2,\t – \t Sputum Color 痰顏色係咩？,N.A.
3,Weight Loss 有冇體重下降？,☑️ No \t ☐ Yes
3.1,\t – \t Weight Loss Amount 輕咗幾多？,N.A.
4,Appetite Loss 胃口差别？,☑️ No \t ☐ Yes
5,Food Preference 咩唔食？,no preference


In [8]:
session_info = requests.post(f"{NGROK_DOMAIN}/get_session_info", json={"session_id": session_id})
session_info = session_info.json()

In [9]:
session_info

{'responses': {'curr_question_id': 'initial_status',
  'grace_uttrs_history': ['initial_status'],
  'history': ['initial_status'],
  'q_tracker': ['initial_status'],
  'session_id': 494016298,
  'user_uttrs_history': []}}